
Load in faces dataset as RDD
Apply transformation to build a new RDD
cache RDD

Present a new face data set and apply transformation to it.

Write a reduce map function to reduce the possible matches.

Build in slight tolerances to reduce errors.

Filter by race
Filter by gender
Filter by eye width with tolerances
Filter by mouth width with tolerances

Ability to add in more filter options.

Return # of matching rows.


genders => (0 = "Male", 1 = "Female")
races => (0 = "White", 2 = "Asian")

Class face {
    // known variables
    imageWidth in pixels
    imageHeight in pixels
    faceWidth as a percentage
    faceHeight as a percentage
    faceCenter as a tuple
    eyeLeft as a tuple
    eyeRight as a tuple
    eyesDistance as a percentage
    mouthLeft as a tuple
    mouthRight as a tuple
    
    xm = float percentage (multiplier of x to scale to [0,1] graph)
    ym = float percentage (multiplier of y to scale to [0,1] graph)
    elsx = float (x coordinate of eye left)
    elsy = float (y coordinate of eye left)
    
    function setCx(){
        
        return this->
    }
    function setXm(){
        $this->xm = .5/cx
    }
    function eyeDistance(){
    
    
    }
    
    funciton mouthWidth(){
    
    }

}

cxm = .5/cx  //gives the multiplier for x of the face
cym = .5/cy //gives the multiplier for y of the face



EyeLeftScaled x and y coordinates on a [0,1] graph
elsx = eyeLeft(x) * (cxm)
elsy = eyeLeft(y) * (cym)

ersx = eyeRight(x) * (cxm)
ersy = eyeRight(y) * (cym)


Distance between eyes

function getEyeDistance {
    eyeRight (x) - EyeLeft (x)

    Example:
    er = (.35, .75) and el = (.75,.75)

    elsx - ersx = edx (eye distance)

    convert back to percentage
    exd = edx / cxm
}

function getMouthWidth {
    
}


In [1]:
class Image (width: Int, height: Int){
    var x: Int = width
    var y: Int = height 
}

class Face (w: Double, h: Double, cX: Double, cY: Double){
    var width: Double = w
    var height: Double = h
    var centerX: Double = cX
    var centerY: Double = cY
    var cxm: Double = .5/centerX
    var cym: Double = .5/centerY 
    var x0: Double = centerX - (width/2)
    var y0: Double = centerY - (height/2)  
}

class Eye (eyeX: Double, eyeY: Double) {
    var x: Double = eyeX
    var y: Double = eyeY
}

class FaceResult (eyeW: Double, mouthW: Double, gen: Double, ra: Double) extends java.io.Serializable {
    var eyeWidth: Double = eyeW
    var mouthWidth: Double = mouthW
    var gender: Double = gen
    var race: Double = ra
}

In [2]:
def transformX (face: Face, x: Double):Double = {
    var xval: Double = x
    return math.abs(xval - face.x0) / face.width
}
def transformY (face:Face, y: Double):Double = {
    var yval: Double = y
    return math.abs(yval - face.y0) / face.height
}

def eyeDistance (face: Face, eyeLeft: Eye, eyeRight: Eye):Double = {
    var el: Tuple2[Double, Double] = (transformX(face, eyeLeft.x), transformY(face, eyeLeft.y))
    var er: Tuple2[Double, Double] = (transformX(face, eyeRight.x), transformY(face, eyeRight.y))
   
    return er._1 - el._1
}
def mouthWidth (face: Face, mouthLeft: Tuple2[Double, Double], mouthRight: Tuple2[Double, Double]):Double = {
    var mouthL: Tuple2[Double, Double] = mouthLeft
    var mouthR: Tuple2[Double, Double] = mouthRight
    var mouthLX: Double = transformX(face, mouthL._1)
    var mouthRX: Double = transformX(face, mouthR._1)
    return mouthRX - mouthLX
}


In [3]:
val faces = sc.textFile("/resources/facesUpdated0.csv")
val faceCount = faces.count()
val facesMap2 = faces.map(line => line.split(","))
def calculate2(line:String): FaceResult = {
    val values = line.split(",")
    val face = new Face(values(30).toDouble, values(28).toDouble, values(13).toDouble, values(14).toDouble)
    val eyeD: Double = eyeDistance(face, new Eye(values(15).toDouble, values(16).toDouble), new Eye(values(17).toDouble, values(18).toDouble))
    val mouthW: Double = mouthWidth(face, (values(20).toDouble, values(21).toDouble), (values(22).toDouble, values(22).toDouble))
    
    val faceResult = new FaceResult(eyeD, mouthW, values(3).toDouble, values(10).toDouble)
    
    return faceResult
}

val facesMap = faces.map(calculate2)
facesMap

MapPartitionsRDD[4] at map at <console>:30

In [4]:
val testFace = sc.textFile("/resources/brandonDetectUpdated0.csv")
val testFaceMap = testFace.map(calculate2)

val testFaceObj = testFaceMap.first()

println(testFaceObj.race, testFaceObj.gender, testFaceObj.eyeWidth, testFaceObj.mouthWidth)

(0.0,0.0,0.04167816829268289,0.030716241463414617)


In [5]:
import org.apache.spark.mllib.clustering.KMeans
import org.apache.spark.mllib.linalg.Vectors

// Load and parse the data
//val data = sc.textFile("data.txt ")
val inputData = Vectors.dense(testFaceObj.race, testFaceObj.gender, testFaceObj.eyeWidth, testFaceObj.mouthWidth)

val parsedData = facesMap.map(s => Vectors.dense(s.race, s.gender, s.eyeWidth, s.mouthWidth))

// Cluster the data into n classes using KMeans
// Set number of clusters equal to the number of data points
val numClusters = 17
val numIterations = 20
val clusters = KMeans.train(parsedData, numClusters, numIterations)

// Evaluate clustering by computing Within Set Sum of Squared Errors
val WSSSE = clusters.computeCost(parsedData)
println("Within Set Sum of Squared Errors = " + WSSSE)

val idx = clusters.predict(inputData)
clusters.clusterCenters(idx)

Within Set Sum of Squared Errors = 0.0


[0.0,0.0,0.04167816829268289,0.030716241463414617]

In [6]:
val facesWomen = facesMap2.filter(array => array(3) == "1")
facesWomen.count()

7

In [7]:
val facesMen = facesMap2.filter(array => array(3) == "0")
facesMen.count()

9

In [8]:
val facesAsianWomen = facesWomen.filter(array => array(10) == "1")
facesAsianWomen.count()

5

In [9]:
val facesAsianMen = facesMen.filter(array => array(10) == "1")
facesAsianMen.count()

4